# Modules 

In [1]:
import numpy as np
import pandas as pd

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 1700)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# from IPython.core.display import display, HTML
# display(HTML("<style>.container { width:100% !important; }</style>"))

from datetime import date

# Data

In [6]:
SSYY = 'SS20' # Season zero

remove_clearance = False

In [7]:
d = str(date.today())
deliverable = 'data/FW20_forecasts_' + d + '.xlsx'
deliverable_cl = 'data/FW20_forecasts_' + d + 'incl_cl.xlsx'

seasons = ['SS17', 'SS18', 'SS19', 'SS20'] # ADD THIS TO file_names TO AVOID MANUAL

# --- list of file names to use ---
if(remove_clearance):
    file_names = ['data/dat_GAS.xlsx', 'data/preds.xlsx', 'data/preds_season_w_pcm.xlsx', 
                  'data/opt_preds_season.xlsx', deliverable, SSYY, seasons]
elif(remove_clearance == False):
    file_names = ['data/dat_GAS_cl.xlsx',  'data/preds_cl.xlsx', 'data/preds_season_w_pcm_cl.xlsx', 
                  'data/opt_preds_season_cl.xlsx', deliverable_cl, SSYY, seasons]
    
file_names = np.array(file_names)
np.save('data/file_names', file_names)

In [8]:
# --- data ---
primary_file = 'data/ch4k_eu.csv'

dat0 = pd.read_csv(primary_file)
dat0 = dat0[dat0.season.isin(seasons)]

# dat0 = dat0[dat0.season == 'FW19'][:10000] # subset for development

# --- clearance/non-clearance --- 
if(remove_clearance):
    dat0['clearance'] = dat0.clearance.fillna(0) 
    dat0['net_qty'] = (1 - dat0.clearance)*dat0.net_qty
    print('clearance_removed')
else:
    print('clearance_included')
    

dat0.to_csv('data/run_data.csv', index=False)

clearance_included


# Articles of interest

In [5]:
SS21 = pd.read_csv('data/SS2021_articles.csv', index_col = 0)
articles_of_interest = dat0[dat0.article_number.isin(SS21['Article Number'])].article_number.unique()
np.save('data/articles_of_interest', articles_of_interest)

In [ ]:
# deprecated article wrangling --- 

# buyer_table = pd.read_csv('data/EU_seasons.csv', low_memory = False, error_bad_lines = False, sep = ",").sort_values(['article_number', 'brand', 'season']) # 26 Aug
# buyer_table = buyer_table[(buyer_table.season == file_names[5])]


# ## adidas  

# adi = pd.read_excel('data/RMA-02_ Market Range Plan_24102019.xlsx', sheet_name='BSO', skiprows=2).drop('x', axis = 1) # adidas RMA2 -- 21 October
# adi = adi[['Article Number', 'WE eCom 26.11.2019', 'Market Retail Price']]
# adi = adi[adi['WE eCom 26.11.2019'] != 0]

# ## rbk  

# rbk = pd.read_excel('data/eCom Range Download_Reebok_FW20.xlsx') 
# rbk = rbk[['Article Number', 'RBK WE eCom', 'Hub Retail Price']]

# clssc_rbk = pd.read_csv('data/reebok_classics.csv').article_number # additional forecastables

# ## stealths

# stealth = pd.read_csv('data/stealth_carryovers_FW2020.csv', low_memory = False, error_bad_lines = False, sep = ";")
# # stealth2 = pd.read_csv('data/stealth_carryovers_eu_RMA2.csv').rename(columns = {'article1': 'article', 'article2': 'stealth_article'})
# # stealth3 = pd.concat([stealth, stealth2])

# # FW19_range = buyer_table.article_number.unique()
# FW20_range = set(adi['Article Number']).union(set(rbk['Article Number']))
# stealths = stealth[stealth.article.isin(FW20_range)].stealth_article # New forecasts: IN FW20 --- NOT IN FW19

# rbk_1920 = pd.read_excel('data/rbk_1920.xlsx').article_number_FW19.unique() # model-based stealths 

# ## Combine all and save

# articles_of_interest = (
#         set(buyer_table.article_number).intersection(set(adi['Article Number'])). # FW19 intersect adi FW20 range
#     union( 
#         set(buyer_table.article_number).intersection(set(rbk['Article Number']))). # FW19 intersect rbk RW20 range
#     union(clssc_rbk).
#     union(stealths).
#     union(rbk_1920)
# )

# # print('Forecastables: ', len(articles_of_interest))
# # print()

# # print('adi carryovers: ', len(set(buyer_table.article_number).intersection(set(adi['Article Number']))))
# # print('rbk carryovers: ', len(set(buyer_table.article_number).intersection(set(rbk['Article Number']))))
# # print('clssc_rbk', len(set(clssc_rbk)))

# articles_of_interest = np.array(articles_of_interest)
# np.save('articles_of_interest', articles_of_interest)

In [ ]:
# deprecated eCom forecasts --- 

# rbk = rbk.rename(columns = {'RBK WE eCom': 'WE eCom'})
# adi = adi.rename(columns = {'WE eCom 26.11.2019': 'WE eCom'})

# ecom = (pd.concat([adi[['Article Number', 'WE eCom']], rbk[['Article Number', 'WE eCom']]]).
#         rename(columns = {'Article Number': 'article_number', 'WE eCom': 'ecom_fc'})
#        )

# ecom.to_excel('data/ecom.xlsx', index=False)